In [1]:
!pip install llama_index==0.9.34
!pip install trafilatura
!pip install bitsandbytes
!pip install accelerate

In [ ]:
import logging
import sys
import torch
import os
#from credentials
#import OPENAI_API_KEY, HF_TOKEN
#os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["TRANSFORMERS_CACHE"] = "../.cache/huggingface/" # set cache dir for transformers
os.environ["LLAMA_INDEX_CACHE_DIR"] = "../.cache/llama_index/" # set cache dir for llama_index



from transformers import BitsAndBytesConfig
from llama_index import set_global_service_context
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate
from llama_index.readers import TrafilaturaWebReader
from llama_index import (
    # GPTVectorStoreIndex,
    VectorStoreIndex,
    ServiceContext,
    download_loader,
    SummaryIndex,
)
from llama_index.response.notebook_utils import display_response


root_url = "https://www.acls.org/fellows-grantees/?_fellow_year=2023&_paged="
urls = [root_url + str(i) for i in range(1, 2)]
documents = TrafilaturaWebReader().load_data(urls)
print("Loaded", len(documents), "documents")


quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

def messages_to_prompt(messages):
  prompt = ""
  for message in messages:
    if message.role == 'system':
      prompt += f"<|system|>\n{message.content}</s>\n"
    elif message.role == 'user':
      prompt += f"<|user|>\n{message.content}</s>\n"
    elif message.role == 'assistant':
      prompt += f"<|assistant|>\n{message.content}</s>\n"

  if not prompt.startswith("<|system|>\n"):
    prompt = "<|system|>\n</s>\n" + prompt
  prompt = prompt + "<|assistant|>\n"

  return prompt

llm = HuggingFaceLLM(
    model_name="stabilityai/stablelm-base-alpha-3b",  # "HuggingFaceH4/zephyr-7b-alpha"
    tokenizer_name="stabilityai/stablelm-base-alpha-3b",      #"HuggingFaceH4/zephyr-7b-alpha"
    query_wrapper_prompt=PromptTemplate("<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"),
    context_window=3000,
    max_new_tokens=1600,
    model_kwargs={"quantization_config": quantization_config},
    # tokenizer_kwargs={},
    generate_kwargs={"temperature": 0.7, "top_k": 50, "do_sample": True},
    messages_to_prompt=messages_to_prompt,
    device_map="auto",
)

service_context = ServiceContext.from_defaults(
  llm=llm,
  embed_model="local:BAAI/bge-large-en",
  chunk_size=512,
  chunk_overlap=64,)
vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)
summary_index = SummaryIndex.from_documents(documents, service_context=service_context)

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

query_engine = vector_index.as_query_engine(response_mode="tree_summarize", similarity_top_k=4)

query = "There are 24 pieces of Fellows and Grantees information. Please list all of awardees and their programs."
response = query_engine.query(query)
print(query)
print(response)

In [ ]:
!pip install accelerate

In [ ]:
import logging
import sys
import torch
import os
#from credentials
#import OPENAI_API_KEY, HF_TOKEN
#os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["TRANSFORMERS_CACHE"] = "../.cache/huggingface/" # set cache dir for transformers
os.environ["LLAMA_INDEX_CACHE_DIR"] = "../.cache/llama_index/" # set cache dir for llama_index
from transformers import BitsAndBytesConfig
from llama_index import set_global_service_context
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate
from llama_index.readers import TrafilaturaWebReader
from llama_index import (
    # GPTVectorStoreIndex,
    VectorStoreIndex,
    ServiceContext,
    download_loader,
    SummaryIndex,
)
from llama_index.response.notebook_utils import display_response


root_url = "https://www.acls.org/fellows-grantees/?_fellow_year=2023&_paged="
urls = [root_url + str(i) for i in range(1, 2)]
documents = TrafilaturaWebReader().load_data(urls)
print("Loaded", len(documents), "documents")


quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

def messages_to_prompt(messages):
  prompt = ""
  for message in messages:
    if message.role == 'system':
      prompt += f"<|system|>\n{message.content}</s>\n"
    elif message.role == 'user':
      prompt += f"<|user|>\n{message.content}</s>\n"
    elif message.role == 'assistant':
      prompt += f"<|assistant|>\n{message.content}</s>\n"

  if not prompt.startswith("<|system|>\n"):
    prompt = "<|system|>\n</s>\n" + prompt
  prompt = prompt + "<|assistant|>\n"

  return prompt

llm = HuggingFaceLLM(
    model_name="HuggingFaceH4/zephyr-7b-alpha",
    tokenizer_name="HuggingFaceH4/zephyr-7b-alpha",
    query_wrapper_prompt=PromptTemplate("<|system|>\n</s>\n<|user|>\n{query_str}</s>\n<|assistant|>\n"),
    context_window=3000,
    max_new_tokens=1600,
    model_kwargs={"quantization_config": quantization_config},
    # tokenizer_kwargs={},
    generate_kwargs={"temperature": 0.7, "top_k": 50, "do_sample": True},
    messages_to_prompt=messages_to_prompt,
    device_map="auto",
)

service_context = ServiceContext.from_defaults(
  llm=llm,
  embed_model="local:BAAI/bge-large-en",
  chunk_size=512,
  chunk_overlap=64,)
vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)
summary_index = SummaryIndex.from_documents(documents, service_context=service_context)

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

query_engine = vector_index.as_query_engine(response_mode="tree_summarize", similarity_top_k=4)

query = "There are 24 pieces of Fellows and Grantees information. Please list all of awardees and their programs."
response = query_engine.query(query)
print(query)
print(response)

In [ ]:
import logging
import sys
import torch
import os
from transformers import BitsAndBytesConfig
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate
from llama_index.readers import TrafilaturaWebReader
from llama_index import VectorStoreIndex, ServiceContext, SummaryIndex

root_url = "https://www.acls.org/fellows-grantees/?_fellow_year=2023&_paged="
# Update the range to include URLs for 5 pages
urls = [root_url + str(i) for i in range(1, 6)]

for page in range(1, 6):
    print("Loading data for page", page)
    documents = TrafilaturaWebReader().load_data([root_url + str(page)])
    print("Loaded", len(documents), "documents for page", page)

    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )

    def messages_to_prompt(messages):
        prompt = ""
        for message in messages:
            if message.role == 'system':
                prompt += f"\n{message.content}</s>\n"
            elif message.role == 'user':
                prompt += f"\n{message.content}</s>\n"
            elif message.role == 'assistant':
                prompt += f"\n{message.content}</s>\n"

        if not prompt.startswith("\n"):
            prompt = "\n</s>\n" + prompt
        prompt = prompt + "\n"

        return prompt

    llm = HuggingFaceLLM(
        model_name="HuggingFaceH4/zephyr-7b-alpha",
        tokenizer_name="HuggingFaceH4/zephyr-7b-alpha",
        query_wrapper_prompt=PromptTemplate("\n</s>\n\n{query_str}</s>\n\n"),
        context_window=3000,
        max_new_tokens=1600,
        model_kwargs={"quantization_config": quantization_config},
        generate_kwargs={"temperature": 0.7, "top_k": 50, "do_sample": True},
        messages_to_prompt=messages_to_prompt,
        device_map="auto",
    )

    service_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model="local:BAAI/bge-large-en",
        chunk_size=512,
        chunk_overlap=64,
    )

    vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)
    summary_index = SummaryIndex.from_documents(documents, service_context=service_context)

    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

    query_engine = vector_index.as_query_engine(response_mode="tree_summarize", similarity_top_k=4)

    query = "There are 24 pieces of Fellows and Grantees information. Please list all of awardees and their programs."
    response = query_engine.query(query)
    print(query)
    print(response)

    # Clear memory
    del documents, quantization_config, llm, service_context, vector_index, summary_index, query_engine, response
    torch.cuda.empty_cache()


Loading data for page 1
Loaded 1 documents for page 1


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


There are 24 pieces of Fellows and Grantees information. Please list all of awardees and their programs.
1. Sarah Aurelia Carson - ACLS Fellowship Program
2. Christine B. Case - ACLS Leading Edge Fellowships
3. Melanie Chambliss - ACLS Fellowship Program
4. Sandy F. Chang - Mellon/ACLS Dissertation Completion Fellowships (2019), ACLS Fellowship Program (2023)
5. Nicola Chávez Courtright - Mellon/ACLS Dissertation Innovation Fellowships
6. Francisco Chen-López - ACLS Project Development Grants
7. Kiara M. Childs - ACLS Leading Edge Fellowships
8. Janessa M. Chinana - Mellon/ACLS Dissertation Innovation Fellowships
9. Dendup Chophel - The Robert H. N. Ho Family Foundation Early Career Research Fellowships in Buddhist Studies, The Robert H. N. Ho Family Foundation Dissertation Fellowships in Buddhist Studies
10. Nicholas Carby Denning - ACLS Fellowship Program
11. Christabel Devadoss - ACLS Project Development Grants
12. Polina Dimova - ACLS Fellowship Program
13. Yuri W. Doolan - ACLS Fe

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for o

KeyboardInterrupt: 

CODE FOR A DIFFERENT MODEL

In [ ]:
import logging
import sys
import torch
from transformers import BitsAndBytesConfig
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate
from llama_index.readers import TrafilaturaWebReader
from llama_index import VectorStoreIndex, ServiceContext, SummaryIndex

root_url = "https://www.acls.org/fellows-grantees/?_fellow_year=2023&_paged="
urls = [root_url + str(i) for i in range(1, 6)]

for page in range(1, 6):
    print("Loading data for page", page)
    documents = TrafilaturaWebReader().load_data([root_url + str(page)])
    print("Loaded", len(documents), "documents for page", page)

    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    )

    def messages_to_prompt(messages):
        prompt = ""
        for message in messages:
            if message.role == 'system':
                prompt += f"\n{message.content}</s>\n"
            elif message.role == 'user':
                prompt += f"\n{message.content}</s>\n"
            elif message.role == 'assistant':
                prompt += f"\n{message.content}</s>\n"

        if not prompt.startswith("\n"):
            prompt = "\n</s>\n" + prompt
        prompt = prompt + "\n"

        return prompt

    llm = HuggingFaceLLM(
        model_name="EleutherAI/gpt-neo-2.7B",
        tokenizer_name="EleutherAI/gpt-neo-2.7B",
        query_wrapper_prompt=PromptTemplate("\n</s>\n\n{query_str}</s>\n\n"),
        context_window=3000,
        max_new_tokens=1600,
        model_kwargs={"quantization_config": quantization_config},
        generate_kwargs={"temperature": 0.7, "top_k": 50, "do_sample": True},
        messages_to_prompt=messages_to_prompt,
        device_map="auto",
    )

    service_context = ServiceContext.from_defaults(
        llm=llm,
        embed_model="local:BAAI/bge-large-en",
        chunk_size=512,
        chunk_overlap=64,
    )

    vector_index = VectorStoreIndex.from_documents(documents, service_context=service_context)
    summary_index = SummaryIndex.from_documents(documents, service_context=service_context)

    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

    query_engine = vector_index.as_query_engine(response_mode="tree_summarize", similarity_top_k=4)

    query = "There are 24 pieces of Fellows and Grantees information. Please list all of awardees and their programs."
    response = query_engine.query(query)
    print(query)
    print(response)

    # Clear memory
    del documents, quantization_config, llm, service_context, vector_index, summary_index, query_engine, response
    torch.cuda.empty_cache()


In [ ]:
from llama_index.llms import HuggingFaceLLM
from llama_index.prompts import PromptTemplate

# Define the prompt template
prompt_template = "https://www.acls.org/fellows-grantees/?_fellow_year=2023&_paged=1"
#Given the ACLS website URL: https://www.acls.org/fellows-grantees/?_fellow_year=2023&_paged=1, write a Python web scraper using BeautifulSoup to extract the grant information. Your scraper should:
#"""

# Define the LLM model and tokenizer
llm = HuggingFaceLLM(model_name="HuggingFaceH4/zephyr-7b-alpha", tokenizer_name="HuggingFaceH4/zephyr-7b-alpha")

# Define the prompt template
prompt_template = PromptTemplate(prompt_template)

# Generate the web scraping code using llm.complete
generated_code = llm.complete(prompt_template, max_tokens=500)

# Print the generated code
print(generated_code)


In [ ]:
import llama_index
print(llama_index.__version__)


0.9.34
